# Creating a Sagemaker Model
The cell below uses the training.py script to train a TensorFlow neural network using Sagemaker. I suggest you run this a few times locally (where your instance_type = 'local') with only a few epochs to make sure you're not going to waste pressious AWS credits and that things are saving in the right places!

Replace the SageMakerRole with your own role (which can be found in the AWS IAM section) and your bucket uirs to S3 paths of your choice. Finally specify your tensorflow and python framework version and you're good to go!

In [29]:
from sagemaker.tensorflow import TensorFlow

SageMakerRole = 'YOUR_SAGEMAKER_ROLE'
bucket_uri = 's3://BUCKET_NAME'
data_uri = bucket_uri + '/data'         # Directory containing data
callback_uri = bucket_uri + '/logs'     # Directory to store the logs
model_uri = bucket_uri + '/models'      # Directory to store the model

instance_type = 'local' #'ml.c5.xlarge'
hyperparameters = {'epochs': 500, 'batch_size': 64, 'learning_rate': 0.01, 'tb_callback': callback_uri, 'sm_model_dir': model_uri}

estimator = TensorFlow(
    entry_point='training.py',
    model_dir=model_uri,
    output_path=bucket_uri,
    instance_type=instance_type,
    instance_count=1,
    hyperparameters=hyperparameters,
    role=SageMakerRole,
    base_job_name='tf-CloudClassification',
    framework_version='2.4',
    py_version='py37'
)

estimator.fit(data_uri)

2021-08-31 06:33:59 Starting - Starting the training job...
2021-08-31 06:34:22 Starting - Launching requested ML instancesProfilerReport-1630391639: InProgress
......
2021-08-31 06:35:23 Starting - Preparing the instances for training......
2021-08-31 06:36:24 Downloading - Downloading input data
2021-08-31 06:36:24 Training - Downloading the training image...
2021-08-31 06:36:44 Training - Training image download completed. Training in progress.2021-08-31 06:36:44.875602: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-08-31 06:36:44.879045: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-08-31 06:36:44.984936: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-08-31 06:36:47,397 sagemaker-training-toolkit INFO     Imported framewo

## Evaluating and Deploying
Running "S3_REGION=YOUR_REGION tensorboard --logdir=s3://PATH_TO_LOGS" in the terminal opens tensorboard on the local host with your chosen metrics.

If you're happy with the results from the model training, it can be deployed using the line below!

In [30]:
predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------!

## Quick test!
Test the predictor on a local image to make sure all is working well before trying to use it in an application.

In [8]:
from sagemaker.tensorflow import TensorFlowPredictor
from PIL import Image
import numpy as np

from image_prep import lbl_switch

endpoint_name = 'MODEL_ENDPOINT'
test_file_path = '../Assets/Veiltest.png'



predictor = TensorFlowPredictor(endpoint_name)
img = Image.open(test_file_path)
img_array = np.asarray(img).reshape(1,125,125,3)

response = predictor.predict(data=img_array)
[predictions] = response['predictions']
print(predictions)
print(f'Model predicted {lbl_switch(np.argmax(predictions), False)} with probability {np.max(predictions)}')

[0.0248658489, 0.0385490395, 0.163259655, 0.115230821, 0.658094645]
Model predicted Veil with probability 0.658094645
